<h1>Data Preparation</h1>
<p>In this section, data for training and testing the model is prepared. The section can be divided as follow:</p>
<ol>
<li>Data loading</li>
<li>Conversion of non-numeric column to numeric</li>
<li>Inversion of dataset order</li>
<li>Creating indicators</li>
<li>Cleaning dataset</li>
<li>Data scaling</li>
<li>Rolling window</li>
<li>Save dataset</li>

</ol>

<h3>1. Data loading</h3>

In [24]:
import pandas as pd
import numpy as np

In [25]:
df_mbl=pd.read_csv("assets/mbl_data.csv",index_col=1,parse_dates=['Date'])
df_mbl.head()

,Symbol,Open,High,Low,Close,Percent Change,Volume
Date,,,,,,,
2024-08-22,MBL,280.0,289.0,277.1,286.2,2.25 %,"284,184.00"
2024-08-21,MBL,281.0,286.0,272.0,279.9,-0.39 %,"345,405.00"
2024-08-18,MBL,280.0,295.0,273.0,281.0,1.08 %,"666,974.00"
2024-08-15,MBL,262.1,282.7,258.1,278.0,8.17 %,"754,130.00"
2024-08-14,MBL,258.0,258.5,248.4,257.0,1.58 %,"167,329.00"


The Date column is parsed as a datetime object and set as index.

<h3>2. Conversion of non-numeric column to numeric</h3>

In [26]:
df_mbl['Volume'] = df_mbl['Volume'].str.replace(',', '', regex=False).astype(float)
df_mbl["Percent Change"] = df_mbl["Percent Change"].str.replace(' %', '', regex=False)
df_mbl["Percent Change"] = pd.to_numeric(df_mbl["Percent Change"], errors='coerce')

The non-numeric columns Volume and Percent Change are converted to numeric values.

In [27]:
df_mbl.head()

,Symbol,Open,High,Low,Close,Percent Change,Volume
Date,,,,,,,
2024-08-22,MBL,280.0,289.0,277.1,286.2,2.25,284184.0
2024-08-21,MBL,281.0,286.0,272.0,279.9,-0.39,345405.0
2024-08-18,MBL,280.0,295.0,273.0,281.0,1.08,666974.0
2024-08-15,MBL,262.1,282.7,258.1,278.0,8.17,754130.0
2024-08-14,MBL,258.0,258.5,248.4,257.0,1.58,167329.0


In [28]:
df_mbl.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1127 entries, 2024-08-22 to 2019-08-25
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Symbol          1127 non-null   object 
 1   Open            1127 non-null   float64
 2   High            1127 non-null   float64
 3   Low             1127 non-null   float64
 4   Close           1127 non-null   float64
 5   Percent Change  1126 non-null   float64
 6   Volume          1127 non-null   float64
dtypes: float64(6), object(1)
memory usage: 70.4+ KB


We can see Volume and Price Change have been converted into float64. But a null value has appeared during conversion. So, using interpolation method, we can fill the null value.

In [29]:
df_mbl['Percent Change']=df_mbl['Percent Change'].interpolate(method='linear')

In [30]:
df_mbl.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1127 entries, 2024-08-22 to 2019-08-25
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Symbol          1127 non-null   object 
 1   Open            1127 non-null   float64
 2   High            1127 non-null   float64
 3   Low             1127 non-null   float64
 4   Close           1127 non-null   float64
 5   Percent Change  1127 non-null   float64
 6   Volume          1127 non-null   float64
dtypes: float64(6), object(1)
memory usage: 70.4+ KB


No null values remaining.

<h3>3. Inversion of dataset order</h3>

The order of the dataset is converted from <i>Future to Past</i> to <i>Past to Future </i> for convenience.

In [31]:
df_mbl=df_mbl.sort_index(ascending=True)

<h3>4. Creating indicators</h3>

<p>Many investors and traders in the stock market use technical indicators as cues for future market trends. Three popular technical indicators are selected here.</p>
<p>The three indicators are:</p>
<ul>
<li>Simple moving average(SMA)</li>
<li>Weighted moving average(WMA)</li>
<li>Relative Strength index(RSI)</li>
</ul>
<p>The three indicators are calculated using the closing prices of the stock.</p>

In [32]:
import pandas_ta as ta

<h4>Moving average</h4>
<p>Simple moving average (SMA) is the unweighted mean value of the specified time period. Weighted moving average (WMA) assigns more weight to the latest data points, since they contain more relevant information than data points in the distant past. Here, the time period of the moving average is taken as 10 days.</p>

In [33]:
sma10=ta.sma(df_mbl["Close"], 10)
sma10.head(15)

Date
2019-08-25      NaN
2019-08-26      NaN
2019-08-27      NaN
2019-08-28      NaN
2019-08-29      NaN
2019-09-01      NaN
2019-09-02      NaN
2019-09-03      NaN
2019-09-04      NaN
2019-09-05    253.4
2019-09-08    253.2
2019-09-09    252.8
2019-09-10    252.4
2019-09-11    252.2
2019-09-12    252.1
Name: SMA_10, dtype: float64

The first 9 rows of the dataset cant have a SMA so they are null.

In [34]:
wma10=ta.wma(df_mbl["Close"],10)
wma10.head(15)

Date
2019-08-25           NaN
2019-08-26           NaN
2019-08-27           NaN
2019-08-28           NaN
2019-08-29           NaN
2019-09-01           NaN
2019-09-02           NaN
2019-09-03           NaN
2019-09-04           NaN
2019-09-05    253.418182
2019-09-08    253.345455
2019-09-09    252.763636
2019-09-10    252.436364
2019-09-11    252.181818
2019-09-12    252.509091
Name: WMA_10, dtype: float64

The first 9 rows of the dataset cant have a WMA so they are null.

<h4>Relative Strength Index (RSI)</h4>
<p>RSI is a momentum indicator that investigates the current and historical gain and losses of the
recent trading period, measured on a scale from 0 to 100.</p>

In [35]:
rsi14=ta.rsi(df_mbl['Close'],14)
rsi14.head(15)

Date
2019-08-25          NaN
2019-08-26          NaN
2019-08-27          NaN
2019-08-28          NaN
2019-08-29          NaN
2019-09-01          NaN
2019-09-02          NaN
2019-09-03          NaN
2019-09-04          NaN
2019-09-05          NaN
2019-09-08          NaN
2019-09-09          NaN
2019-09-10          NaN
2019-09-11          NaN
2019-09-12    51.122955
Name: RSI_14, dtype: float64

The length of RSI is taken as 14 since it is the most typically used timeframe.

In [36]:
df_mbl['SMA10']=sma10
df_mbl['WMA10']=wma10
df_mbl['RSI14']=rsi14

Adding indicators to the dataset.

In [37]:
df_mbl.head()

,Symbol,Open,High,Low,Close,Percent Change,Volume,SMA10,WMA10,RSI14
Date,,,,,,,,,,
2019-08-25,MBL,257.0,257.0,252.0,255.0,-0.78,13021.0,NaN,NaN,NaN
2019-08-26,MBL,257.0,257.0,251.0,254.0,-0.39,30391.0,NaN,NaN,NaN
2019-08-27,MBL,259.0,259.0,254.0,255.0,0.39,21428.0,NaN,NaN,NaN
2019-08-28,MBL,256.0,256.0,252.0,253.0,-0.78,12203.0,NaN,NaN,NaN
2019-08-29,MBL,253.0,255.0,247.0,255.0,0.79,40974.0,NaN,NaN,NaN


There are null values which requires handling.

<h3>5. Cleaning dataset</h3>

In [38]:
df_clean = df_mbl.dropna()
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1113 entries, 2019-09-12 to 2024-08-22
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Symbol          1113 non-null   object 
 1   Open            1113 non-null   float64
 2   High            1113 non-null   float64
 3   Low             1113 non-null   float64
 4   Close           1113 non-null   float64
 5   Percent Change  1113 non-null   float64
 6   Volume          1113 non-null   float64
 7   SMA10           1113 non-null   float64
 8   WMA10           1113 non-null   float64
 9   RSI14           1113 non-null   float64
dtypes: float64(9), object(1)
memory usage: 95.6+ KB


The rows with null values are dropped. Thus, first 13 rows are removed from the dataset.

In [39]:
df_clean=df_clean.iloc[:,1:]
df_clean.head()

,Open,High,Low,Close,Percent Change,Volume,SMA10,WMA10,RSI14
Date,,,,,,,,,
2019-09-12,252.0,255.0,251.0,254.0,1.20,16686.0,252.1,252.509091,51.122955
2019-09-15,255.0,255.0,246.0,248.0,-2.36,18710.0,252.0,251.763636,40.857728
2019-09-16,245.0,248.0,242.0,245.0,-1.21,18061.0,252.0,250.490909,36.871202
2019-09-17,249.0,252.0,249.0,251.0,2.45,10694.0,251.7,250.309091,47.834025
2019-09-18,253.0,255.0,250.0,252.0,0.40,7967.0,251.1,250.363636,49.410857


We have obtained a clean dataset containing the indicators.

In [40]:
df_clean.to_csv("assets/clean_data.csv",index=False)

<h3>6. Data scaling</h3>

In [41]:
from sklearn.preprocessing import StandardScaler
import pickle
scaler=StandardScaler()
scaler.fit(df_clean)
df_normalized=scaler.transform(df_clean)
df_normalized=pd.DataFrame(df_normalized,columns=df_clean.columns)
df_normalized.describe()

,Open,High,Low,Close,Percent Change,Volume,SMA10,WMA10,RSI14
count,1.113000e+03,1.113000e+03,1.113000e+03,1.113000e+03,1.113000e+03,1.113000e+03,1113.000000,1.113000e+03,1113.000000
mean,2.553613e-16,-1.532168e-16,1.021445e-16,-4.085780e-16,5.586028e-18,-2.553613e-17,0.000000,7.150116e-16,0.000000
std,1.000450e+00,1.000450e+00,1.000450e+00,1.000450e+00,1.000450e+00,1.000450e+00,1.000450,1.000450e+00,1.000450
min,-1.641119e+00,-1.605938e+00,-1.612313e+00,-1.615589e+00,-4.861564e+00,-8.249563e-01,-1.547393,-1.569328e+00,-2.174425
25%,-6.058594e-01,-5.884778e-01,-5.788565e-01,-5.815372e-01,-5.246544e-01,-5.725008e-01,-0.583096,-5.805488e-01,-0.712778
50%,-3.336862e-01,-3.263302e-01,-3.282176e-01,-3.440757e-01,-2.383193e-02,-3.474775e-01,-0.328485,-3.304260e-01,-0.017057
75%,5.162287e-01,5.092653e-01,5.370016e-01,5.316686e-01,4.290394e-01,1.754262e-01,0.574536,5.905132e-01,0.632641
max,3.234620e+00,3.196278e+00,3.043390e+00,3.157218e+00,5.293411e+00,8.449557e+00,2.983327,3.001802e+00,2.772318


Through the min max scaling process, each feature component is normalized to the range [0,1], since the range of values of raw data varies widely, and it helps gradient descent to converge much faster.

In [42]:
import joblib
joblib.dump(scaler, "assets/scaler.gz") 

['assets/scaler.gz']

The scaler is stored for future usage.

In [43]:
df_normalized.to_csv("assets/clean_normalized_data.csv",index=False)

<h3>7. Rolling window</h3>

In [44]:
from helper.rolling_window import rolling_window

rolling_dataset = rolling_window(df_normalized,120)
print("Shape of dataset: {number_of_windows} X {window_size} X {num_features}".format(number_of_windows=len(rolling_dataset),window_size=rolling_dataset[0].shape[0],num_features=rolling_dataset[0].shape[1]))

Shape of dataset: 994 X 120 X 9


Through a rolling window technique, we can create a dataset of historical data points that can be used to train a machine learning model for predictions.</br> The shape of the dataset is 1103 X 11 X 9, where 1103 is the number of windows, 11 is the length of a window, and 9 is the number of features.

<h3>8. Save dataset</h3>

In [45]:
pd.concat(rolling_dataset).to_csv("assets/rolling_window_data.csv",index=False)

The dataset is saved for training and testing the model.